# Baseline PSNR evaluation (portable)

This notebook evaluates the baseline `on-the-fly-nvs` pipeline on the held-out test set.

- Works both on Google Colab and local Windows/Linux/macOS.
- Paths are resolved dynamically; ensure the baseline model outputs are under `on-the-fly-nvs/results/scene`.
- CSV results are read/written under `results/scene/metrics_testhold_{TEST_HOLD}.csv`. 

In [ ]:
#@title 1) Clone and install on-the-fly-nvs + dependencies
!git clone --recursive https://github.com/graphdeco-inria/on-the-fly-nvs.git
%cd on-the-fly-nvs

!pip install -q -r requirements.txt

import torch
print("Torch:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Cloning into 'on-the-fly-nvs'...
remote: Enumerating objects: 808, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 808 (delta 22), reused 14 (delta 13), pack-reused 773 (from 1)
Receiving objects: 100% (808/808), 5.20 MiB | 19.28 MiB/s, done.
Resolving deltas: 100% (259/259), done.
Submodule 'submodules/Depth-Anything-V2' (https://github.com/DepthAnything/Depth-Anything-V2.git) registered for path 'submodules/Depth-Anything-V2'
Submodule 'submodules/fused-ssim' (https://github.com/rahul-goel/fused-ssim) registered for path 'submodules/fused-ssim'
Submodule 'submodules/graphdecoviewer' (https://github.com/graphdeco-inria/graphdecoviewer.git) registered for path 'submodules/graphdecoviewer'
Cloning into '/content/on-the-fly-nvs/submodules/Depth-Anything-V2'...
remote: Enumerating objects: 142, done.        
remote: Total 142 (delta 0), reused 0 (delta 0), pack-reused 142 (from 1)        
Receiving objects: 100% (142/142),

In [ ]:
!pip -q install opencv-python-headless tqdm

import os, shutil, glob, re, json, subprocess, shlex, struct
from pathlib import Path
import numpy as np, cv2

# Portable paths: Colab vs local

def is_colab():
    return "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ

ROOT = Path.cwd()
REPO = "/content/on-the-fly-nvs" if is_colab() else str((ROOT / "on-the-fly-nvs").resolve())

SCENE_DIR  = f"{REPO}/data/my_scene"     
MODEL_DIR  = f"{REPO}/results/scene"    
TEST_HOLD  = 30                          

for d in [MODEL_DIR, SCENE_DIR]:
    os.makedirs(d, exist_ok=True)

In [5]:
# @title 1) Créer test/ par sous-échantillonnage (1 image / TEST_HOLD)
from pathlib import Path
import re

images_dir = Path(SCENE_DIR) / "images"
test_dir   = Path(SCENE_DIR) / "test"
images_dir.mkdir(parents=True, exist_ok=True)

# Si un dataset sort de structure "frame-000000.color.jpg", on peut normaliser en ".jpg"
# sans écraser les originaux: on copie/symlink si besoin.
def canonical_name(p: Path) -> str:
    s = p.name
    if s.endswith(".color.jpg"):
        s = s.replace(".color.jpg", ".jpg")
    return s

if not test_dir.exists():
    test_dir.mkdir(parents=True, exist_ok=True)
    imgs = sorted([p for p in images_dir.iterdir() if p.suffix.lower() in [".jpg",".jpeg",".png"] or p.name.endswith(".color.jpg")])
    assert len(imgs)>0, f"Aucune image trouvée dans {images_dir}"
    sel = [imgs[i] for i in range(0, len(imgs), TEST_HOLD)]
    print(f"Création de {len(sel)} vues de test dans: {test_dir}")
    for p in sel:
        dst = test_dir / canonical_name(p)
        if not dst.exists():
            # copie (plus robuste que symlink sur Colab)
            dst.write_bytes(p.read_bytes())
else:
    print(f"'test/' existe déjà ({test_dir}).")


Création de 51 vues de test dans: /content/on-the-fly-nvs/data/my_scene/test


In [ ]:
#@title 3) Launch incremental training (repo default bootstrap)
import subprocess, shlex, os, textwrap, sys

# Let the repo do its internal bootstrap (~8 frames).
env = os.environ.copy()
prev_pp = env.get("PYTHONPATH", "")
extra = os.getcwd()
env["PYTHONPATH"] = os.pathsep.join([p for p in [prev_pp, extra] if p])

cmd = f"python train.py -s {SCENE_DIR} -m {MODEL_DIR} --test_hold {TEST_HOLD} --viewer_mode none --downsampling 2.5  --save_every 100"
print(cmd)
ret = subprocess.call(shlex.split(cmd), env=env)

print("Exit code:", ret)
assert ret == 0, "Training interrupted. Check logs above."


python train.py -s /content/on-the-fly-nvs/data/my_scene -m /content/on-the-fly-nvs/results/scene --test_hold 30 --viewer_mode none --downsampling 2.5  --save_every 100
Exit code: 0


In [ ]:
# @title 3) Verify sets (identical filenames)
import os, glob

GT_TEST_DIR     = f"{SCENE_DIR}/test"
RENDER_TEST_DIR = f"{MODEL_DIR}/test_images"

gt_names = sorted([os.path.basename(p) for p in glob.glob(os.path.join(GT_TEST_DIR, "*")) if os.path.isfile(p)])
rd_names = sorted([os.path.basename(p) for p in glob.glob(os.path.join(RENDER_TEST_DIR, "*")) if os.path.isfile(p)])

print(f"GT test images: {len(gt_names)}")
print(f"Rendered tests: {len(rd_names)}")

missing_in_render = [n for n in gt_names if n not in rd_names]
missing_in_gt     = [n for n in rd_names if n not in gt_names]

if not missing_in_render and not missing_in_gt:
    print("✅ Name sets match (GT test ↔ renders).")
else:
    if missing_in_render:
        print(f"⚠️ Missing in renders ({len(missing_in_render)}) e.g.: {missing_in_render[:5]}")
    if missing_in_gt:
        print(f"⚠️ Missing in GT test ({len(missing_in_gt)}) e.g.: {missing_in_gt[:5]}")

GT test images : 51
Rendered tests : 51
✅ Les ensembles de noms correspondent (GT test ↔ rendus).


In [ ]:
# @title 4) PSNR/SSIM/LPIPS (paper) + CSV
!pip -q install lpips scikit-image

import os, glob, cv2, numpy as np, torch
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
import lpips
import math

GT_TEST_DIR     = f"{SCENE_DIR}/test"
RENDER_TEST_DIR = f"{MODEL_DIR}/test_images"
SCENE_IS_TUM    = False    # True => TUM mask (GT pixels > 0) for PSNR
CSV_PATH        = f"{MODEL_DIR}/metrics_testhold_{TEST_HOLD}.csv"

device = "cuda" if torch.cuda.is_available() else "cpu"
lpips_net = lpips.LPIPS(net='vgg').to(device).eval()  # Official LPIPS (VGG)

def read_rgb(path):
    im = cv2.imread(path, cv2.IMREAD_COLOR)
    if im is None: return None
    return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

def psnr_tensor(img, gt, mask=None):
    """
    img, gt: torch.FloatTensor [C,H,W] in [0,1]
    mask   : torch.BoolTensor [H,W] optional (e.g., TUM)
    Return: PSNR(dB) with MAX=1 => -10*log10(MSE)
    """
    if mask is not None:
        diff = (img - gt)[:, mask]
    else:
        diff = (img - gt).reshape(3, -1)
    mse = torch.mean(diff**2)
    if mse <= 1e-12:
        return float("inf")
    return float(-10.0 * torch.log10(mse).item())

def ssim_Y(gt_rgb_u8, rd_rgb_u8):
    """SSIM on luminance (Y) in 8-bit sRGB."""
    gt_y = cv2.cvtColor(gt_rgb_u8, cv2.COLOR_RGB2YCrCb)[:,:,0]
    rd_y = cv2.cvtColor(rd_rgb_u8, cv2.COLOR_RGB2YCrCb)[:,:,0]
    return float(ssim(gt_y, rd_y, data_range=255))

def lpips_dist(gt_rgb_u8, rd_rgb_u8):
    """LPIPS (VGG) on RGB normalized to [-1,1], shape [1,3,H,W]."""
    gt_t = torch.from_numpy(np.ascontiguousarray(gt_rgb_u8)).permute(2,0,1).float()/127.5 - 1.0
    rd_t = torch.from_numpy(np.ascontiguousarray(rd_rgb_u8)).permute(2,0,1).float()/127.5 - 1.0
    with torch.no_grad():
        d = lpips_net(gt_t.unsqueeze(0).to(device), rd_t.unsqueeze(0).to(device)).item()
    return float(d)

# Strict pairing by filename
gt_files = sorted([p for p in glob.glob(os.path.join(GT_TEST_DIR, "*")) if os.path.isfile(p)])
assert gt_files, f"No images in {GT_TEST_DIR}"

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/vgg.pth
Paires trouvées: 51 / 51


Metrics: 100%|██████████| 51/51 [00:29<00:00,  1.71it/s]


Moyennes sur 51 vues held-out (test_hold=30) :
  PSNR  : 20.74 dB  | médiane 20.78  | min 9.01 | max 28.46
  SSIM(Y): 0.595    | médiane 0.602 | min 0.339 | max 0.834
  LPIPS : 0.648    | médiane 0.641 | min 0.486 | max 0.815

Exemples :
  frame-000000.jpg : PSNR=27.02 dB | SSIM(Y)=0.717 | LPIPS=0.622
  frame-000030.jpg : PSNR=26.94 dB | SSIM(Y)=0.637 | LPIPS=0.548
  frame-000060.jpg : PSNR=26.01 dB | SSIM(Y)=0.659 | LPIPS=0.604
  frame-000090.jpg : PSNR=27.05 dB | SSIM(Y)=0.692 | LPIPS=0.563
  frame-000120.jpg : PSNR=17.23 dB | SSIM(Y)=0.556 | LPIPS=0.653

CSV écrit : /content/on-the-fly-nvs/results/scene/metrics_testhold_30.csv


In [ ]:
import os
import pandas as pd
from pathlib import Path

# Resolve CSV path dynamically (works on Colab and local)
try:
    csv_path = f"{MODEL_DIR}/metrics_testhold_{TEST_HOLD}.csv"
except NameError:
    # Fallback if previous cells were not run
    def is_colab():
        return "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ
    ROOT = Path.cwd()
    REPO = "/content/on-the-fly-nvs" if is_colab() else str((ROOT / "on-the-fly-nvs").resolve())
    MODEL_DIR = f"{REPO}/results/scene"
    TEST_HOLD = 30
    csv_path = f"{MODEL_DIR}/metrics_testhold_{TEST_HOLD}.csv"

print("Reading:", csv_path)
df = pd.read_csv(csv_path)
df

,name,psnr_db,ssim_y,lpips_vgg
0,frame-000000.jpg,27.0238,0.717464,0.622084
1,frame-000030.jpg,26.9380,0.636804,0.548166
2,frame-000060.jpg,26.0054,0.659029,0.603682
3,frame-000090.jpg,27.0474,0.691582,0.562514
4,frame-000120.jpg,17.2347,0.555778,0.652683
5,frame-000150.jpg,28.4063,0.690032,0.603873
6,frame-000180.jpg,26.1707,0.643667,0.581015
7,frame-000210.jpg,20.7887,0.580250,0.610618
8,frame-000240.jpg,27.0383,0.625515,0.611514
9,frame-000270.jpg,25.1707,0.590733,0.601547
